In [1]:
import pandas as pd
import numpy as np



In [2]:
# Read census files
df_1900 = pd.read_csv('data/lakeland_1900_census.csv')
df_1920 = pd.read_csv('data/lakeland_1920_census.csv')
df_1930 = pd.read_csv('data/lakeland_1930_census.csv')
df_1940 = pd.read_csv('data/lakeland_1940_census.csv')
df_1950 = pd.read_csv('data/lakeland_1950_census.csv')

# Print sample of each DataFrame
print("\nSample of each census year:")
for year, df in [('1900', df_1900), ('1920', df_1920), 
                 ('1930', df_1930), ('1940', df_1940),
                 ('1950', df_1950)]:
    print(f"\n{year} Census Columns:", df.columns.tolist())




Sample of each census year:

1900 Census Columns: ['pk', 'dwelling', 'family', 'last_name', 'first_name', 'head_last', 'head_first', 'relation_head', 'race', 'sex', 'age', 'marital', 'place_birth', 'work', 'owned_rented']

1920 Census Columns: ['pk', 'dwelling number', 'family', 'last_name', 'first_name', 'relation_head', 'head_last', 'head_first', 'sex', 'race', 'marital', 'age', 'place_birth', 'work', 'business', 'owned_rented']

1930 Census Columns: ['pk', 'dwelling number', 'family', 'street_name', 'last_name', 'first_name', 'relation_head', 'sex', 'race', 'marital', 'age', 'place_birth', 'work', 'business', 'owned_rented']

1940 Census Columns: ['pk', 'ed', 'house_num', 'street_name', 'last_name', 'first_name', 'relation_head', 'head_last', 'head_first', 'sex', 'race', 'marital', 'age', 'place_birth', 'work', 'business', 'owned_rented']

1950 Census Columns: ['pk', 'ed', 'house_num', 'build_num', 'street_name', 'last_name', 'first_name', 'relation_head', 'head_last', 'head_first'

In [3]:
def standardize_columns(df, year):
    df = df.copy()
    
    # Add census year
    df['census_year'] = year
    
    # Rename columns to match database schema
    column_mapping = {
        'pk': 'source_pk',
        'first_name': 'first_name',
        'last_name': 'last_name',
        'dwelling': 'dwelling',
        'family': 'family',
        'relation_head': 'relation_to_hoh',
        'head_first': 'hoh_first_name',
        'head_last': 'hoh_last_name',
        'sex': 'sex',
        'race': 'race',
        'age': 'age',
        'marital': 'marital_status',
        'work': 'work',
        'business': 'business',
        'owned_rented': 'owned_rented'
    }
    
    # Rename existing columns
    df = df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns})
    
    # Add missing columns with null values
    for col in column_mapping.values():
        if col not in df.columns:
            df[col] = None
            
    return df

In [4]:
# ... existing code ...
df_1900 = standardize_columns(df_1900, 1900)
df_1920 = standardize_columns(df_1920, 1920)
df_1930 = standardize_columns(df_1930, 1930)
df_1940 = standardize_columns(df_1940, 1940)
df_1950 = standardize_columns(df_1950, 1950)


In [5]:
combined_df = pd.concat([
    df_1900, df_1920, df_1930, df_1940, df_1950
], ignore_index=True)

In [6]:
race_mapping = {
    'W': 'White',
    'B': 'Black',
    'N': 'Black',
    'Neg': 'Black'
}
combined_df['race'] = combined_df['race'].map(race_mapping).fillna(combined_df['race'])

In [7]:
marital_mapping = {
    'M': 'Married',
    'S': 'Single',
    'W': 'Widowed',
    'Wd': 'Widowed',
    'D': 'Divorced',
    'Mar': 'Married',
    'Sep': 'Separated',
    'Nev': 'Never Married'
}
combined_df['marital_status'] = combined_df['marital_status'].map(marital_mapping).fillna(combined_df['marital_status'])

In [8]:
final_columns = [
    'source_pk', 'census_year', 'person_id', 'record_id', 'attribute_id', 'family_id',
    'first_name', 'last_name', 'dwelling', 'family', 'relation_to_hoh',
    'hoh_first_name', 'hoh_last_name', 'sex', 'race', 'age',
    'marital_status', 'work', 'business', 'owned_rented'
]

In [9]:
# Add the missing ID columns
combined_df['person_id'] = np.arange(len(combined_df))
combined_df['record_id'] = np.arange(len(combined_df))
combined_df['attribute_id'] = np.arange(len(combined_df))
combined_df['family_id'] = combined_df.groupby(['census_year', 'dwelling', 'family']).ngroup()

# Now select the final columns
final_df = combined_df[final_columns]
final_df

,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented
0,1,1900,0,0,0,0.0,Ezra,Vanvalkenburg,32,32.0,Head,Ezra,Vanvalkenburg,M,White,58,Married,Merchant,None,O
1,2,1900,1,1,1,0.0,Hattie,Vanvalkenburg,32,32.0,Wife,Ezra,Vanvalkenburg,F,White,56,Married,NaN,None,NaN
2,3,1900,2,2,2,0.0,William,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,M,White,22,Single,Bakes Bread,None,NaN
3,4,1900,3,3,3,0.0,Jay,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,M,White,20,Single,Baker,None,NaN
4,5,1900,4,4,4,0.0,Jessie,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,F,White,15,Single,At School,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,310,1950,792,792,792,NaN,Pearl,Cameron,None,NaN,Wife,Andrew,Cameron,NaN,Black,39,Married,General Housework,Private Family,None
793,311,1950,793,793,793,NaN,Charles A.,Lomax,None,NaN,Head,Charles A,Lomax,NaN,Black,47,Married,Janitor,Hardware,None
794,312,1950,794,794,794,NaN,Etlka,Lomax,None,NaN,Wife,Charles A,Lomax,NaN,Black,46,Married,General Housework,Private Families,None
795,313,1950,795,795,795,NaN,Dervey A.,Lomax,None,NaN,Son,Charles A,Lomax,NaN,Black,24,Never Married,Janitor,Bank,None


In [10]:
final_df

,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented
0,1,1900,0,0,0,0.0,Ezra,Vanvalkenburg,32,32.0,Head,Ezra,Vanvalkenburg,M,White,58,Married,Merchant,None,O
1,2,1900,1,1,1,0.0,Hattie,Vanvalkenburg,32,32.0,Wife,Ezra,Vanvalkenburg,F,White,56,Married,NaN,None,NaN
2,3,1900,2,2,2,0.0,William,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,M,White,22,Single,Bakes Bread,None,NaN
3,4,1900,3,3,3,0.0,Jay,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,M,White,20,Single,Baker,None,NaN
4,5,1900,4,4,4,0.0,Jessie,Vanvalkenburg,32,32.0,Son,Ezra,Vanvalkenburg,F,White,15,Single,At School,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,310,1950,792,792,792,NaN,Pearl,Cameron,None,NaN,Wife,Andrew,Cameron,NaN,Black,39,Married,General Housework,Private Family,None
793,311,1950,793,793,793,NaN,Charles A.,Lomax,None,NaN,Head,Charles A,Lomax,NaN,Black,47,Married,Janitor,Hardware,None
794,312,1950,794,794,794,NaN,Etlka,Lomax,None,NaN,Wife,Charles A,Lomax,NaN,Black,46,Married,General Housework,Private Families,None
795,313,1950,795,795,795,NaN,Dervey A.,Lomax,None,NaN,Son,Charles A,Lomax,NaN,Black,24,Never Married,Janitor,Bank,None


In [11]:
final_df.to_csv('data/lakeland_combined_census.csv', index=False)

In [12]:
combined_df = pd.read_csv('data/lakeland_combined_census.csv')
combined_df

,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented
0,1,1900,0,0,0,0.0,Ezra,Vanvalkenburg,32.0,32.0,Head,Ezra,Vanvalkenburg,M,White,58,Married,Merchant,NaN,O
1,2,1900,1,1,1,0.0,Hattie,Vanvalkenburg,32.0,32.0,Wife,Ezra,Vanvalkenburg,F,White,56,Married,NaN,NaN,NaN
2,3,1900,2,2,2,0.0,William,Vanvalkenburg,32.0,32.0,Son,Ezra,Vanvalkenburg,M,White,22,Single,Bakes Bread,NaN,NaN
3,4,1900,3,3,3,0.0,Jay,Vanvalkenburg,32.0,32.0,Son,Ezra,Vanvalkenburg,M,White,20,Single,Baker,NaN,NaN
4,5,1900,4,4,4,0.0,Jessie,Vanvalkenburg,32.0,32.0,Son,Ezra,Vanvalkenburg,F,White,15,Single,At School,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,310,1950,792,792,792,NaN,Pearl,Cameron,NaN,NaN,Wife,Andrew,Cameron,NaN,Black,39,Married,General Housework,Private Family,NaN
793,311,1950,793,793,793,NaN,Charles A.,Lomax,NaN,NaN,Head,Charles A,Lomax,NaN,Black,47,Married,Janitor,Hardware,NaN
794,312,1950,794,794,794,NaN,Etlka,Lomax,NaN,NaN,Wife,Charles A,Lomax,NaN,Black,46,Married,General Housework,Private Families,NaN
795,313,1950,795,795,795,NaN,Dervey A.,Lomax,NaN,NaN,Son,Charles A,Lomax,NaN,Black,24,Never Married,Janitor,Bank,NaN


In [13]:
# Find duplicates based on first and last name
duplicates = combined_df[combined_df.duplicated(subset=['first_name', 'last_name'], keep=False)]

# Sort by first and last name to group duplicates together
duplicates = duplicates.sort_values(['first_name', 'last_name'])

duplicates


,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented
475,54,1940,475,475,475,NaN,Agnes,Forrest,NaN,NaN,Head,Agnes,Forrest,F,Black,63,Widowed,Domestic,Private Home,O
633,151,1950,633,633,633,NaN,Agnes,Forrest,NaN,NaN,Grandmother,Thomas,Randall,F,Black,75,Widowed,NaN,NaN,NaN
464,43,1940,464,464,464,NaN,Amelia,Wilson,NaN,NaN,Granddaughter,William,Gross,F,Black,3,Single,NaN,NaN,NaN
606,124,1950,606,606,606,NaN,Amelia,Wilson,NaN,NaN,Gand Daughter,William,gross,F,Black,13,Never Married,NaN,NaN,NaN
366,181,1920,366,366,366,NaN,Anderson,Walls,NaN,124.0,Son,Hattie,Walls,M,Black,7,Single,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,149,1950,631,631,631,NaN,Thomas,Randall,NaN,NaN,Son,Thomas,Randall,M,Black,4,Never Married,NaN,NaN,NaN
455,34,1940,455,455,455,NaN,William,Gross,NaN,NaN,Head,William,Gross,M,Black,63,Married,Janitor,Pullman Yard,O
591,109,1950,591,591,591,NaN,William,Gross,NaN,NaN,Head,William,Gross,M,Black,72,Widowed,NaN,NaN,NaN
418,47,1930,418,418,418,NaN,William P,Brown,NaN,NaN,Nephew In Law,NaN,NaN,M,Black,22,Married,Chauffeur,Private Family,NaN


In [14]:

# Convert age to numeric, setting non-numeric values to NaN
duplicates['age'] = pd.to_numeric(duplicates['age'], errors='coerce')
duplicates


,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented
475,54,1940,475,475,475,NaN,Agnes,Forrest,NaN,NaN,Head,Agnes,Forrest,F,Black,63.0,Widowed,Domestic,Private Home,O
633,151,1950,633,633,633,NaN,Agnes,Forrest,NaN,NaN,Grandmother,Thomas,Randall,F,Black,75.0,Widowed,NaN,NaN,NaN
464,43,1940,464,464,464,NaN,Amelia,Wilson,NaN,NaN,Granddaughter,William,Gross,F,Black,3.0,Single,NaN,NaN,NaN
606,124,1950,606,606,606,NaN,Amelia,Wilson,NaN,NaN,Gand Daughter,William,gross,F,Black,13.0,Never Married,NaN,NaN,NaN
366,181,1920,366,366,366,NaN,Anderson,Walls,NaN,124.0,Son,Hattie,Walls,M,Black,7.0,Single,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,149,1950,631,631,631,NaN,Thomas,Randall,NaN,NaN,Son,Thomas,Randall,M,Black,4.0,Never Married,NaN,NaN,NaN
455,34,1940,455,455,455,NaN,William,Gross,NaN,NaN,Head,William,Gross,M,Black,63.0,Married,Janitor,Pullman Yard,O
591,109,1950,591,591,591,NaN,William,Gross,NaN,NaN,Head,William,Gross,M,Black,72.0,Widowed,NaN,NaN,NaN
418,47,1930,418,418,418,NaN,William P,Brown,NaN,NaN,Nephew In Law,NaN,NaN,M,Black,22.0,Married,Chauffeur,Private Family,NaN


In [15]:
duplicates.to_csv('data/lakeland_duplicates.csv', index=False)


In [16]:
duplicates
# Find duplicates where first name, last name AND age are the same
exact_duplicates = duplicates[duplicates.duplicated(subset=['first_name', 'last_name', 'age'], keep=False)]

# Sort by first and last name to group duplicates together
exact_duplicates = exact_duplicates.sort_values(['first_name', 'last_name', 'age'])

exact_duplicates



,source_pk,census_year,person_id,record_id,attribute_id,family_id,first_name,last_name,dwelling,family,relation_to_hoh,hoh_first_name,hoh_last_name,sex,race,age,marital_status,work,business,owned_rented


In [17]:
def clean_census_data(df):
    df = df.copy()
    
    # Generate IDs first
    df['person_id'] = np.arange(len(df))  # Unique for each person
    df['record_id'] = df.groupby('census_year').ngroup()  # Same for census year
    df['attribute_id'] = df.groupby('race').ngroup()  # Group by race (Black, White, etc.)
    df['family_id'] = df.groupby(['census_year', 'dwelling', 'family']).ngroup()
    
    # Standardize mappings
    mappings = {
        'sex': {
            'M': 'Male', 'm': 'Male', 'F': 'Female', 'f': 'Female',
            'N': 'Unknown', 'S': 'Unknown', 'D': 'Unknown'
        },
        'race': {
            'neg': 'Black', 'Unknown': 'Unknown', 'Black': 'Black', 'White': 'White'
        },
        'marital_status': {
            'Wed': 'Married', 'Mer': 'Married', 'Nen': 'Never Married',
            'Div': 'Divorced', 'Married': 'Married', 'Single': 'Single',
            'Widowed': 'Widowed', 'Never Married': 'Never Married'
        },
        'owned_rented': {
            'O': 'Owner', 'R': 'Renter', 'Owner': 'Owner', 'Renter': 'Renter'
        }
    }
    
    # Apply mappings
    for col, mapping in mappings.items():
        if col in df.columns:
            df[col] = df[col].map(mapping).fillna('Unknown')
    
    # Convert numeric columns
    numeric_columns = ['dwelling', 'family']  # Removed IDs since we generate them
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0).astype(int)
    
    # Convert age to numeric
    df['age'] = pd.to_numeric(df['age'], errors='coerce').fillna(0).astype(int)
    
    # Fill nulls for string columns
    string_columns = ['work', 'business', 'relation_to_hoh', 'hoh_first_name', 'hoh_last_name']
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].fillna('None' if col in ['work', 'business'] else 'Unknown')
    
    return df

# Use the function
combined_df = clean_census_data(combined_df)

In [18]:
# Export the cleaned combined dataframe to CSV
combined_df.to_csv('data/lakeland_combined_census_cleaned.csv', index=False)
